# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 25 2022 11:33AM,242652,21,621414,"NBTY Global, Inc.",Released
1,Jul 25 2022 11:19AM,242651,10,8321037-SD,Eywa Pharma Inc.,Released
2,Jul 25 2022 11:04AM,242644,15,8319672,"Carwin Pharmaceutical Associates, LLC",Released
3,Jul 25 2022 11:04AM,242644,15,8319673,"Carwin Pharmaceutical Associates, LLC",Released
4,Jul 25 2022 11:03AM,242643,10,SO94314,"Senseonics, Incorporated",Released
5,Jul 25 2022 11:00AM,242637,15,8319727,"Brookfield Pharmaceuticals, LLC",Released
6,Jul 25 2022 11:00AM,242637,15,8320241,"Brookfield Pharmaceuticals, LLC",Released
7,Jul 25 2022 11:00AM,242637,15,8320242,"Brookfield Pharmaceuticals, LLC",Released
8,Jul 25 2022 11:00AM,242637,15,8320243,"Brookfield Pharmaceuticals, LLC",Released
9,Jul 25 2022 11:00AM,242637,15,8320244,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,242638,Released,1
28,242643,Released,1
29,242644,Released,2
30,242651,Released,1
31,242652,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242638,NaN,NaN,1.0
242643,NaN,NaN,1.0
242644,NaN,NaN,2.0
242651,NaN,NaN,1.0
242652,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242576,0.0,0.0,1.0
242579,0.0,0.0,1.0
242583,0.0,0.0,1.0
242591,0.0,3.0,2.0
242602,6.0,6.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242576,0,0,1
242579,0,0,1
242583,0,0,1
242591,0,3,2
242602,6,6,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242576,0,0,1
1,242579,0,0,1
2,242583,0,0,1
3,242591,0,3,2
4,242602,6,6,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242576,,,1
1,242579,,,1
2,242583,,,1
3,242591,,3,2
4,242602,6,6,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc."
1,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.
2,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC"
4,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated"
5,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC"
29,Jul 25 2022 10:53AM,242638,10,ISDIN Corporation
30,Jul 25 2022 10:43AM,242636,15,"Alliance Pharma, Inc."
31,Jul 25 2022 10:40AM,242635,22,"NBTY Global, Inc."
32,Jul 25 2022 10:39AM,242632,15,"Alliance Pharma, Inc."
95,Jul 25 2022 10:38AM,242634,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",,,1
1,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,,,1
2,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",,,2
3,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",,,1
4,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",,,24
5,Jul 25 2022 10:53AM,242638,10,ISDIN Corporation,,,1
6,Jul 25 2022 10:43AM,242636,15,"Alliance Pharma, Inc.",,,1
7,Jul 25 2022 10:40AM,242635,22,"NBTY Global, Inc.",,1,
8,Jul 25 2022 10:39AM,242632,15,"Alliance Pharma, Inc.",,,63
9,Jul 25 2022 10:38AM,242634,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",1,,
1,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,1,,
2,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",2,,
3,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",1,,
4,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",24,,
5,Jul 25 2022 10:53AM,242638,10,ISDIN Corporation,1,,
6,Jul 25 2022 10:43AM,242636,15,"Alliance Pharma, Inc.",1,,
7,Jul 25 2022 10:40AM,242635,22,"NBTY Global, Inc.",,1,
8,Jul 25 2022 10:39AM,242632,15,"Alliance Pharma, Inc.",63,,
9,Jul 25 2022 10:38AM,242634,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",1,,
1,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,1,,
2,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",2,,
3,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",1,,
4,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN,NaN
3,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0,0.0
3,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4367053,225.0,6.0,6.0
15,970549,90.0,0.0,0.0
16,242634,1.0,0.0,0.0
19,727842,5.0,3.0,0.0
21,485282,2.0,0.0,0.0
22,242635,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4367053,225.0,6.0,6.0
1,15,970549,90.0,0.0,0.0
2,16,242634,1.0,0.0,0.0
3,19,727842,5.0,3.0,0.0
4,21,485282,2.0,0.0,0.0
5,22,242635,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,225.0,6.0,6.0
1,15,90.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,5.0,3.0,0.0
4,21,2.0,0.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,225.0
1,15,Released,90.0
2,16,Released,1.0
3,19,Released,5.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,0.0,3.0,0.0,1.0
Released,225.0,90.0,1.0,5.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,0.0,3.0,0.0,1.0
2,Released,225.0,90.0,1.0,5.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,0.0,3.0,0.0,1.0
2,Released,225.0,90.0,1.0,5.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()